In [3]:
import h5py
import pandas as pd
import numpy as np
import os

In [4]:
def populate_tables(data_folder_name='./data/', 
                   tracers= [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,
                            21,22,23,24,25,26,27, 28, 29,30,31,32,33,34,35,
                            36, 37, 38,  39,40,41,42,43,44,45]): 
    
    #########################################
    """
    This function create the txt files with the strain rate data
    
    Arguments:
    - data_folder_name (str): folder where the raw h5 files are stored. 
        Default is 'outputs-fixed-values'.
    - tracers (str): name of tracers. Default is 'lag'.
    
    Returns:
    - Folder called txt_outputs with all the files.
    """
    #########################################
    
    # Check if the data folder exists
    if not os.path.exists(data_folder_name):
        raise FileNotFoundError(f"Folder '{data_folder_name}' not found.")
    
    # count number of times, checking the number that files that starts with prefix
    prefix = f'tip{tracers[0]}_velocitygradient'
    files = os.listdir(data_folder_name)
    num_files = sum([1 for file in files if file.startswith(prefix)])
    print(num_files)
    # loop by number of files
    time = 0
    #print('num files:', num_files)
    while time < num_files:
        print(f'time: {time}')
        #########################################
        #  LOOP by tracer #
        #########################################
        # create a variable name to store coord
        dataset_name = f'np_array_{time}'
        globals()[dataset_name] = []
        #print('dataset_name', dataset_name)
        for tracer in tracers:
            ## COORDINATES ##
            # open Coordinates H5 
            h5_file_coord =  f'./{data_folder_name}/tip{tracer}-{time}.h5'
            if not os.path.exists(h5_file_coord):
                    print(f"Warning: H5 file not found at '{h5_file_coord}'. Skipping this time step.")
                    time += 1922
                    continue

            with h5py.File(h5_file_coord, 'r') as file:
                for key in file.keys():
                    # create a string with the name of the var, and assign numpy value from h5
                    globals()[dataset_name].append(file[key][:][0])
                    #print('ds_name',globals()[dataset_name])
                

        # create a table for each time and populate them with numpy array 
        # first create the name of the table
        df_name_by_t = f'df_t_{time}'
        # create the table
        globals()[df_name_by_t] = pd.DataFrame(globals()[dataset_name], columns=['x', 'y', 'z'])
        #add index in first column
        globals()[df_name_by_t].insert(0, 'index', globals()[df_name_by_t].index)
        #prit('df_name_by_t', globals()[df_name_by_t])
        

        ## Velocity Gradient TENSOR ##
        # create a variable name for storing strain rate tensor array
        dataset_name = f'np_array_{time}'
        globals()[dataset_name] = []
        for tracer in tracers:
            # open the Strain Rate Tensor H5 files 
            h5_file_path =  f'./{data_folder_name}/tip{tracer}_velocitygradient-{time}.h5'
            if not os.path.exists(h5_file_path):
                    print(f"Warning: H5 Strain Rate Tensor file not found at '{h5_file_path}'. Skipping this time step.")
                    time += 1
                    continue

            with h5py.File(h5_file_path, 'r') as file:
                for key in file.keys():
                    globals()[dataset_name].append(file[key][:][0])
                    #print('ds_name_velgrad',globals()[dataset_name])
                    #print('ds_name_velgrad[0]',globals()[dataset_name][0])
                    #print('testing')
                    #print(globals()[dataset_name][0][0])

                    
                    
            ## temperature
            h5_file_path_temp =  f'./{data_folder_name}/tip{tracer}_temperature-{time}.h5'
            if h5_file_path_temp == f'./{data_folder_name}/tip30_temperature-2353.h5':
                h5_file_path_temp = f'./{data_folder_name}/tip30_temperature-2352.h5'
                
            with h5py.File(h5_file_path_temp, 'r') as file:
                for key in file.keys():
                    # create a string with the name of the var, and assign numpy value from h5
                    temp = file[key][:][0]
                    
            # [a00,a01,a02,a10,a11,a12,a20,a21,a22]
            #print('tracer',tracer)
            tensor_index = tracer - 1
            globals()[df_name_by_t].loc[tensor_index, 'VG00'] = globals()[dataset_name][tensor_index][0]
            globals()[df_name_by_t].loc[tensor_index, 'VG01'] = globals()[dataset_name][tensor_index][1]
            globals()[df_name_by_t].loc[tensor_index, 'VG02'] = globals()[dataset_name][tensor_index][2]
            globals()[df_name_by_t].loc[tensor_index, 'VG10'] = globals()[dataset_name][tensor_index][3]
            globals()[df_name_by_t].loc[tensor_index, 'VG11'] = globals()[dataset_name][tensor_index][4]
            globals()[df_name_by_t].loc[tensor_index, 'VG12'] = globals()[dataset_name][tensor_index][5]
            globals()[df_name_by_t].loc[tensor_index, 'VG20'] = globals()[dataset_name][tensor_index][6]
            globals()[df_name_by_t].loc[tensor_index, 'VG21'] = globals()[dataset_name][tensor_index][7]
            globals()[df_name_by_t].loc[tensor_index, 'VG22'] = globals()[dataset_name][tensor_index][8]
            
            globals()[df_name_by_t].loc[tensor_index, 'TempKelvin'] = temp
            #print('fi')
     
            
            #print('Time',time)
            #print('globals()[df_name_by_t]', globals()[df_name_by_t])
        ## save the df as txt
        output_folder = 'txt_outputs_test_temp'
        output_file = os.path.join(output_folder, f'results{df_name_by_t}.txt')

        # check if the folder exists; if not, create it
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            print('out folder: ', output_folder)

        # save to tab-separated values (.txt) file
        globals()[df_name_by_t].to_csv(output_file, sep='\t', index=False)

        #########


        #print(f'hi {df_name_by_t}') 
        #print(globals()[df_name_by_t].tail())
        time += 1
        
            
        

In [5]:
populate_tables()

1694
time: 0
out folder:  txt_outputs_test_temp
time: 1
time: 2
time: 3
time: 4
time: 5
time: 6
time: 7
time: 8
time: 9
time: 10
time: 11
time: 12
time: 13
time: 14
time: 15
time: 16
time: 17
time: 18
time: 19
time: 20
time: 21
time: 22
time: 23
time: 24
time: 25
time: 26
time: 27
time: 28
time: 29
time: 30
time: 31
time: 32
time: 33
time: 34
time: 35
time: 36
time: 37
time: 38
time: 39
time: 40
time: 41
time: 42
time: 43
time: 44
time: 45
time: 46
time: 47
time: 48
time: 49
time: 50
time: 51
time: 52
time: 53
time: 54
time: 55
time: 56
time: 57
time: 58
time: 59
time: 60
time: 61
time: 62
time: 63
time: 64
time: 65
time: 66
time: 67
time: 68
time: 69
time: 70
time: 71
time: 72
time: 73
time: 74
time: 75
time: 76
time: 77
time: 78
time: 79
time: 80
time: 81
time: 82
time: 83
time: 84
time: 85
time: 86
time: 87
time: 88
time: 89
time: 90
time: 91
time: 92
time: 93
time: 94
time: 95
time: 96
time: 97
time: 98
time: 99
time: 100
time: 101
time: 102
time: 103
time: 104
time: 105
time: 106


In [5]:
# temperature in kelvin
def populate_tables(data_folder_name='./data', 
                   tracers= [63]): # 38, 48, 51, 59
    #########################################
    """
    This function create the txt files with the strain rate data
    
    Arguments:
    - data_folder_name (str): folder where the raw h5 files are stored. 
        Default is 'outputs-fixed-values'.
    - tracers (str): name of tracers. Default is 'lag'.
    
    Returns:
    - Folder called txt_outputs with all the files.
    """
    #########################################
    
    # Check if the data folder exists
    if not os.path.exists(data_folder_name):
        raise FileNotFoundError(f"Folder '{data_folder_name}' not found.")
    
    # count number of times, checking the number that files that starts with prefix
    prefix = f'tip{tracers[0]}_velocitygradient'
    files = os.listdir(data_folder_name)
    num_files = sum([1 for file in files if file.startswith(prefix)])
    print(num_files)
    # loop by number of files
    time = 0
    #print('num files:', num_files)
    while time < num_files:
        print(f'time: {time}')
        #########################################
        #  LOOP by tracer #
        #########################################
        # create a variable name to store coord
        dataset_name = f'np_array_{time}'
        globals()[dataset_name] = []
        #print('dataset_name', dataset_name)
        for tracer in tracers:
            ## COORDINATES ##
            # open Coordinates H5 
            h5_file_coord =  f'./{data_folder_name}/tip{tracer}-{time}.h5'
            if not os.path.exists(h5_file_coord):
                    print(f"Warning: H5 file not found at '{h5_file_coord}'. Skipping this time step.")
                    time += 1222
                    continue

            with h5py.File(h5_file_coord, 'r') as file:
                for key in file.keys():
                    # create a string with the name of the var, and assign numpy value from h5
                    #print('file[key][:][0]', file[key][:][0])
                    #print('tracer', tracer)
                    globals()[dataset_name].append(file[key][:][0])
                    #print('ds_name',globals()[dataset_name])
                

        # create a table for each time and populate them with numpy array 
        # first create the name of the table
        df_name_by_t = f'df_t_{time}'
        # create the table
        globals()[df_name_by_t] = pd.DataFrame(globals()[dataset_name], columns=['x', 'y', 'z'])
        #add index in first column
        globals()[df_name_by_t].insert(0, 'index', globals()[df_name_by_t].index)
        #prit('df_name_by_t', globals()[df_name_by_t])
        

        ## Velocity Gradient TENSOR ##
        # create a variable name for storing strain rate tensor array
        dataset_name = f'np_array_{time}'
        globals()[dataset_name] = []
        for tracer in tracers:
            # open the Strain Rate Tensor H5 files 
            h5_file_path =  f'./{data_folder_name}/tip{tracer}_velocitygradient-{time}.h5'
            if not os.path.exists(h5_file_path):
                    print(f"Warning: H5 Strain Rate Tensor file not found at '{h5_file_path}'. Skipping this time step.")
                    time += 1
                    continue

            with h5py.File(h5_file_path, 'r') as file:
                for key in file.keys():
                    globals()[dataset_name].append(file[key][:][0])
                    #print('ds_name_velgrad',globals()[dataset_name])
                    #print('ds_name_velgrad[0]',globals()[dataset_name][0])
                    #print('testing')
                    #print(globals()[dataset_name][0][0])
                    
            ## temperature
            #h5_file_path_temp =  f'./{data_folder_name}/tip{tracer}_temperature-{time}.h5'
            #if h5_file_path_temp == f'./{data_folder_name}/tip30_temperature-2353.h5':
                #h5_file_path_temp = f'./{data_folder_name}/tip30_temperature-2352.h5'
                
            #with h5py.File(h5_file_path_temp, 'r') as file:
                #for key in file.keys():
                    # create a string with the name of the var, and assign numpy value from h5
                     #temp = file[key][:][0]
    
            
                    
            # [a00,a01,a02,a10,a11,a12,a20,a21,a22]
            #print('tracer',tracer)
            tensor_index = 0#tracer - 1 # index starts at 0
            #print(' globals()[dataset_name][tensor_index]',  globals()[dataset_name][tensor_index])
            globals()[df_name_by_t].loc[tensor_index, 'VG00'] = globals()[dataset_name][tensor_index][0]
            globals()[df_name_by_t].loc[tensor_index, 'VG01'] = globals()[dataset_name][tensor_index][1]
            globals()[df_name_by_t].loc[tensor_index, 'VG02'] = globals()[dataset_name][tensor_index][2]
            globals()[df_name_by_t].loc[tensor_index, 'VG10'] = globals()[dataset_name][tensor_index][3]
            globals()[df_name_by_t].loc[tensor_index, 'VG11'] = globals()[dataset_name][tensor_index][4]
            globals()[df_name_by_t].loc[tensor_index, 'VG12'] = globals()[dataset_name][tensor_index][5]
            globals()[df_name_by_t].loc[tensor_index, 'VG20'] = globals()[dataset_name][tensor_index][6]
            globals()[df_name_by_t].loc[tensor_index, 'VG21'] = globals()[dataset_name][tensor_index][7]
            globals()[df_name_by_t].loc[tensor_index, 'VG22'] = globals()[dataset_name][tensor_index][8]
            
            #globals()[df_name_by_t].loc[tensor_index, 'TempKelvin'] = temp
            #print('fi')
            

            
            #print('Time',time)
            #print('globals()[df_name_by_t]', globals()[df_name_by_t])
        ## save the df as txt
        output_folder = 'txt_outputs_test_63'
        output_file = os.path.join(output_folder, f'results{df_name_by_t}.txt')

        # check if the folder exists; if not, create it
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            print('out folder: ', output_folder)

        # save to tab-separated values (.txt) file
        globals()[df_name_by_t].to_csv(output_file, sep='\t', index=False)

        #########


        #print(f'hi {df_name_by_t}') 
        #print(globals()[df_name_by_t].tail())
        time += 1
        
            
        

In [6]:
populate_tables()

4008
time: 0
out folder:  txt_outputs_test_63
time: 1
time: 2
time: 3
time: 4
time: 5
time: 6
time: 7
time: 8
time: 9
time: 10
time: 11
time: 12
time: 13
time: 14
time: 15
time: 16
time: 17
time: 18
time: 19
time: 20
time: 21
time: 22
time: 23
time: 24
time: 25
time: 26
time: 27
time: 28
time: 29
time: 30
time: 31
time: 32
time: 33
time: 34
time: 35
time: 36
time: 37
time: 38
time: 39
time: 40
time: 41
time: 42
time: 43
time: 44
time: 45
time: 46
time: 47
time: 48
time: 49
time: 50
time: 51
time: 52
time: 53
time: 54
time: 55
time: 56
time: 57
time: 58
time: 59
time: 60
time: 61
time: 62
time: 63
time: 64
time: 65
time: 66
time: 67
time: 68
time: 69
time: 70
time: 71
time: 72
time: 73
time: 74
time: 75
time: 76
time: 77
time: 78
time: 79
time: 80
time: 81
time: 82
time: 83
time: 84
time: 85
time: 86
time: 87
time: 88
time: 89
time: 90
time: 91
time: 92
time: 93
time: 94
time: 95
time: 96
time: 97
time: 98
time: 99
time: 100
time: 101
time: 102
time: 103
time: 104
time: 105
time: 106
ti

time: 857
time: 858
time: 859
time: 860
time: 861
time: 862
time: 863
time: 864
time: 865
time: 866
time: 867
time: 868
time: 869
time: 870
time: 871
time: 872
time: 873
time: 874
time: 875
time: 876
time: 877
time: 878
time: 879
time: 880
time: 881
time: 882
time: 883
time: 884
time: 885
time: 886
time: 887
time: 888
time: 889
time: 890
time: 891
time: 892
time: 893
time: 894
time: 895
time: 896
time: 897
time: 898
time: 899
time: 900
time: 901
time: 902
time: 903
time: 904
time: 905
time: 906
time: 907
time: 908
time: 909
time: 910
time: 911
time: 912
time: 913
time: 914
time: 915
time: 916
time: 917
time: 918
time: 919
time: 920
time: 921
time: 922
time: 923
time: 924
time: 925
time: 926
time: 927
time: 928
time: 929
time: 930
time: 931
time: 932
time: 933
time: 934
time: 935
time: 936
time: 937
time: 938
time: 939
time: 940
time: 941
time: 942
time: 943
time: 944
time: 945
time: 946
time: 947
time: 948
time: 949
time: 950
time: 951
time: 952
time: 953
time: 954
time: 955
time: 956


time: 1640
time: 1641
time: 1642
time: 1643
time: 1644
time: 1645
time: 1646
time: 1647
time: 1648
time: 1649
time: 1650
time: 1651
time: 1652
time: 1653
time: 1654
time: 1655
time: 1656
time: 1657
time: 1658
time: 1659
time: 1660
time: 1661
time: 1662
time: 1663
time: 1664
time: 1665
time: 1666
time: 1667
time: 1668
time: 1669
time: 1670
time: 1671
time: 1672
time: 1673
time: 1674
time: 1675
time: 1676
time: 1677
time: 1678
time: 1679
time: 1680
time: 1681
time: 1682
time: 1683
time: 1684
time: 1685
time: 1686
time: 1687
time: 1688
time: 1689
time: 1690
time: 1691
time: 1692
time: 1693
time: 1694
time: 1695
time: 1696
time: 1697
time: 1698
time: 1699
time: 1700
time: 1701
time: 1702
time: 1703
time: 1704
time: 1705
time: 1706
time: 1707
time: 1708
time: 1709
time: 1710
time: 1711
time: 1712
time: 1713
time: 1714
time: 1715
time: 1716
time: 1717
time: 1718
time: 1719
time: 1720
time: 1721
time: 1722
time: 1723
time: 1724
time: 1725
time: 1726
time: 1727
time: 1728
time: 1729
time: 1730

time: 2419
time: 2420
time: 2421
time: 2422
time: 2423
time: 2424
time: 2425
time: 2426
time: 2427
time: 2428
time: 2429
time: 2430
time: 2431
time: 2432
time: 2433
time: 2434
time: 2435
time: 2436
time: 2437
time: 2438
time: 2439
time: 2440
time: 2441
time: 2442
time: 2443
time: 2444
time: 2445
time: 2446
time: 2447
time: 2448
time: 2449
time: 2450
time: 2451
time: 2452
time: 2453
time: 2454
time: 2455
time: 2456
time: 2457
time: 2458
time: 2459
time: 2460
time: 2461
time: 2462
time: 2463
time: 2464
time: 2465
time: 2466
time: 2467
time: 2468
time: 2469
time: 2470
time: 2471
time: 2472
time: 2473
time: 2474
time: 2475
time: 2476
time: 2477
time: 2478
time: 2479
time: 2480
time: 2481
time: 2482
time: 2483
time: 2484
time: 2485
time: 2486
time: 2487
time: 2488
time: 2489
time: 2490
time: 2491
time: 2492
time: 2493
time: 2494
time: 2495
time: 2496
time: 2497
time: 2498
time: 2499
time: 2500
time: 2501
time: 2502
time: 2503
time: 2504
time: 2505
time: 2506
time: 2507
time: 2508
time: 2509

time: 3189
time: 3190
time: 3191
time: 3192
time: 3193
time: 3194
time: 3195
time: 3196
time: 3197
time: 3198
time: 3199
time: 3200
time: 3201
time: 3202
time: 3203
time: 3204
time: 3205
time: 3206
time: 3207
time: 3208
time: 3209
time: 3210
time: 3211
time: 3212
time: 3213
time: 3214
time: 3215
time: 3216
time: 3217
time: 3218
time: 3219
time: 3220
time: 3221
time: 3222
time: 3223
time: 3224
time: 3225
time: 3226
time: 3227
time: 3228
time: 3229
time: 3230
time: 3231
time: 3232
time: 3233
time: 3234
time: 3235
time: 3236
time: 3237
time: 3238
time: 3239
time: 3240
time: 3241
time: 3242
time: 3243
time: 3244
time: 3245
time: 3246
time: 3247
time: 3248
time: 3249
time: 3250
time: 3251
time: 3252
time: 3253
time: 3254
time: 3255
time: 3256
time: 3257
time: 3258
time: 3259
time: 3260
time: 3261
time: 3262
time: 3263
time: 3264
time: 3265
time: 3266
time: 3267
time: 3268
time: 3269
time: 3270
time: 3271
time: 3272
time: 3273
time: 3274
time: 3275
time: 3276
time: 3277
time: 3278
time: 3279

time: 3973
time: 3974
time: 3975
time: 3976
time: 3977
time: 3978
time: 3979
time: 3980
time: 3981
time: 3982
time: 3983
time: 3984
time: 3985
time: 3986
time: 3987
time: 3988
time: 3989
time: 3990
time: 3991
time: 3992
time: 3993
time: 3994
time: 3995
time: 3996
time: 3997
time: 3998
time: 3999
time: 4000
time: 4001
time: 4002
time: 4003
time: 4004
time: 4005
time: 4006
time: 4007


In [ ]:
!pwd

In [8]:
h5_file_coord =  f'./data/tip5_temperature-786.h5'

with h5py.File(h5_file_coord, 'r') as file:
            for key in file.keys():
                # create a string with the name of the var, and assign numpy value from h5
                 dataset_name = file[key][:][0]
                    

dataset_name

array([1841.6063], dtype=float32)